# NIH Grant & Funding Data Dashboard  

## *What do health science researchers want to know about the Grants & Funding from National Institutes of Health?*

Insert NIH Graphs Here

Begun as a one-room Laboratory of Hygiene in 1887, the National Institutes of Health (NIH) today is one of the world’s foremost medical research centers. It is also the largest public funder of biomedical research in the world, investing more than $32 billion a year to enhance life, and reduce illness and disability. Yet, the devil is in the details. The painstaking process of grant application and review has long been questioned by many researchers that spent more time writing up proposals than doing actual research. With a blogpost on 'the inequality in NIH funding distribution' published last year by the NIH office, more questions regarding how the money was spent were brought up by the researchers. In this project, we attempt to answer some of their key questions through data analysis and visualization using a web-based dashboard. 

Insert screenshots of Comments Here 

## Overview 

Our analysis contains four aspects: 

INSERT POWERPOINT HERE

The technical components involved are: 
- SQL Database interaction
- Complex visualization with Plotly
- Web-based application with flask + Dash
- Data collection and cleaning (some through API request)

We would be using the following packages and libraries: 

In [1]:
# importing dependencies 
import os
import glob
import regex as re
import json
from urllib.request import urlopen
import requests
import time
from geopy import Nominatim
import sqlite3
# dash
import dash
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

# plotting and df
import plotly
print(plotly.__version__)
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

import io
import PIL
r = np.random.RandomState(42)


5.13.0


## Geospatial Analysis  

We first take a look at this data:  
NIH Awards: by project names, prpoject number (Activity code included) Location, Organization, admin IC, funding mechanisms, direct costs, and indirect costs per fiscal year and project number 
https://report.nih.gov/award/index.cfm?ot=&fy=2023&state=&ic=&fm=&orgid=&distr=&rfa=&om=n&pid=#tab5 
 

To create a map visualization, we would be needing the relevant FIPS code for states and counties.  

We import a city FIPS table for geospatial data display. The following dataset maps county (all upper cases) to their FIPs, because county FIPS are needed to visualize the county map in plotly. 

In [2]:
cfips = pd.read_csv("county-fips.csv")
cfips["full_name"] = (cfips["county_name"]+" "+cfips["state_name"]).str.upper()
fip_dict = dict(zip(list(cfips["full_name"]),list(cfips["fips"])))
list(fip_dict.items())[1:5]


[('BALDWIN COUNTY ALABAMA', 1003),
 ('BARBOUR COUNTY ALABAMA', 1005),
 ('BIBB COUNTY ALABAMA', 1007),
 ('BLOUNT COUNTY ALABAMA', 1009)]

And the following table is capable of mapping city to their respective county. 

In [23]:
# just downloaded the county and city fips code from us census data and mapped
city = pd.read_csv("uscities.csv")
city.city = city.city.str.upper()
city.county_name = city.county_name.str.upper()
city.state_name = city.state_name.str.upper()
city = city.dropna()
# store data in database data.db table name "decade"
try:
    conn = sqlite3.connect("data.db",if_exists=False)
    city.to_sql("city",conn,index=False)
except:
    print("The city table already existed.")
conn.close()


The city table already existed.


However, since this table does not contain information on all cities, we make further use of the geolocator module to fill in the missing information. 

In [24]:
# retrieving location: longitude and latitude based on city and state

geolocator = Nominatim(user_agent="app")


The following would be a dict to map state full name to state two-letter code. Plotly has an option to display state map by state code (two letters). 

In [3]:
anym = pd.read_csv("acronynm.csv")
c_dict = dict(zip(list(anym.state.str.upper()),list(anym.code)))

Now we fill in some additional information to make mapping geo codes and future visualization easier:
- MONTH: derive from award notice date
- YEAR: derive from award notice data also
- CITY, STATE: convert to upper
- FULL_LOC: combine city and state, make searching with geo_locator easier
- CODE: state code

In [4]:

data = pd.DataFrame()
path = "/Users/linlin/Desktop/2023/16b/dash/awardsbyloc"
xls_list = os.listdir(path)
xlsx = glob.glob(os.path.join(path, "*.xls"))
for x in xlsx:
    if '.DS_Store' not in x:
        print(x)
        df = pd.read_excel(x)
        df.columns = [s.upper() for s in list(df.columns)]
        df["MONTH"] = pd.to_datetime(df["AWARD NOTICE DATE"]).dt.month
        df["YEAR"] = pd.to_datetime(df["AWARD NOTICE DATE"]).dt.year
        df["CITY"] = df["CITY"].str.upper()
        df["STATE OR COUNTRY NAME"] = df["STATE OR COUNTRY NAME"].str.upper()
        df["FULL_LOC"] = df["CITY"] + " " + df["STATE OR COUNTRY NAME"]
        df['CODE'] = df["STATE OR COUNTRY NAME"].map(c_dict)
        
        if len(data) == 0:
            data = df
        else:
            data = pd.concat([data, df],    # Combine vertically
                            ignore_index = False,
                            sort = False)

        

/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2018.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2019.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2021.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2020.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2022.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2012.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2013.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2014.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2015.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2017.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2016.xls


We retain only the relevant columns in the dataset.

In [8]:
data = data[['ORGANIZATION NAME', 'ORGANIZATION ID (IPF)', 'PROJECT NUMBER',
       'FUNDING MECHANISM', 'PI NAME', 'PI PERSON ID',
       'PROJECT TITLE', 'DIRECT COST', 'INDIRECT COST', 'FUNDING',
       'CITY', 'STATE OR COUNTRY NAME',
       'INSTITUTION TYPE', 'AWARD NOTICE DATE','MONTH', 'YEAR',
       'FULL_LOC', 'CODE']]

In [17]:
data = data[data.YEAR.notna()]
data.shape

(611898, 18)

The different funding mechanisms and institution types of NIH Project and Funding Grants are: 

In [18]:
data['INSTITUTION TYPE'] = data['INSTITUTION TYPE'].fillna('None')

<ipython-input-18-9c351738c920>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['INSTITUTION TYPE']= data['INSTITUTION TYPE'].fillna('None')


In [19]:
data['INSTITUTION TYPE'].unique()

array(['None', 'Research Institutes', 'Domestic Higher Education',
       'Independent Hospitals'], dtype=object)

In [20]:
data['FUNDING MECHANISM'].unique()

array(['RPGs - SBIR/STTR', 'RPGs - Non SBIR/STTR',
       'Other Research-Related', 'Training - Individual',
       'Training - Institutional', 'Research Centers', 'Other',
       'Construction'], dtype=object)

INSERT EXPLANATION TABLE

Since we have a huge amount of data —— over 600k entries, it would be wise to store and query the data using databases. 

In [21]:
# store data in database data.db table name "decade"

conn = sqlite3.connect("data.db")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS decade")
conn.commit()
data.to_sql("decade",conn,index=False)

611898

Now, it's time to write a query function to select data by fiscal year, funding mechanism, and insitution type. Additionally, we would want to map the data to corresponding geocodes, and fill in anything that is missing with the geolocator module.  (The functions are saved in separate python files as required.)

Let's write a small helper function to print all tables in a given database, too.

In [38]:
def print_table(db = 'data.db'):
    conn = sqlite3.connect(db)
    alltbl = """SELECT name FROM sqlite_master  
    WHERE type='table';"""
    cur = conn.cursor()
    print(cur.execute(alltbl).fetchall())
    conn.close()
print_table()

[('city',), ('decade',)]


In [40]:
import numpy as np
import sqlite3
import pandas as pd
FME = ['RPGs - SBIR/STTR',
 'RPGs - Non SBIR/STTR',
 'Other Research-Related',
 'Training - Individual',
 'Training - Institutional',
 'Research Centers',
 'Other',
 'Construction']
INST = ['None',
 'Research Institutes',
 'Domestic Higher Education',
 'Independent Hospitals']
def map_que(year=list(np.arange(2012,2022)),month=list(np.arange(1,12)),
            fme = FME,
            inst = INST):
        cmd = '''
        SELECT D.[PROJECT NUMBER], D.[FUNDING MECHANISM],D.[DIRECT COST],D.[INDIRECT COST],
        D.FUNDING,D.CITY,D.[STATE OR COUNTRY NAME],D.[INSTITUTION TYPE],D.YEAR,D.MONTH,
        D.FULL_LOC,D.CODE,COUNTY_FIPS,C.COUNTY_NAME,C.LAT,C.LNG
        FROM 
                (SELECT LNG,LAT,COUNTY_NAME,CITY,COUNTY_FIPS,STATE_ID
                FROM city) AS C
        LEFT JOIN decade as D
        ON D.CITY = C.CITY
        WHERE D.YEAR IN ''' + str(tuple(year))\
        + ''' AND MONTH IN ''' + str(tuple(month))\
        + ''' AND D.[FUNDING MECHANISM] IN ''' + str(tuple(inst))\
        + ''' AND D.[INSTITUTION TYPE] IN ''' + str(tuple(fme))
        print(cmd)
        conn = sqlite3.connect("data.db")
        cursor = conn.cursor()

        cursor.execute(cmd)
        df = pd.DataFrame(cursor.fetchall(),columns=["PROJECT NUMBER", "FUNDING MECHANISM","DIRECT COST","INDIRECT COST",
        "FUNDING","CITY","STATE OR COUNTRY NAME","INSTITUTION TYPE","YEAR","MONTH",
        "FULL_LOC","CODE","COUNTY_FIPS","COUNTY_NAME","LAT","LNG"])
        print(df.head())
        conn.close()
        return df
df = map_que() 



        SELECT D.[PROJECT NUMBER], D.[FUNDING MECHANISM],D.[DIRECT COST],D.[INDIRECT COST],
        D.FUNDING,D.CITY,D.[STATE OR COUNTRY NAME],D.[INSTITUTION TYPE],D.YEAR,D.MONTH,
        D.FULL_LOC,D.CODE,COUNTY_FIPS,C.COUNTY_NAME,C.LAT,C.LNG
        FROM 
                (SELECT LNG,LAT,COUNTY_NAME,CITY,COUNTY_FIPS,STATE_ID
                FROM city) AS C
        LEFT JOIN decade as D
        ON D.CITY = C.CITY
        WHERE D.YEAR IN (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021) AND MONTH IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11) AND D.[FUNDING MECHANISM] IN ('None', 'Research Institutes', 'Domestic Higher Education', 'Independent Hospitals') AND D.[INSTITUTION TYPE] IN ('RPGs - SBIR/STTR', 'RPGs - Non SBIR/STTR', 'R&D Contracts', 'Other Research-Related', 'Training - Individual', 'Training - Institutional', 'Research Centers', 'Other', 'Construction')
Empty DataFrame
Columns: [PROJECT NUMBER, FUNDING MECHANISM, DIRECT COST, INDIRECT COST, FUNDING, CITY, STATE OR COUNTR

In [54]:
list(data['INSTITUTION TYPE'].unique())

['None',
 'Research Institutes',
 'Domestic Higher Education',
 'Independent Hospitals']

In [49]:
cmd = """
SELECT D.[PROJECT NUMBER], D.[FUNDING MECHANISM],D.[DIRECT COST],D.[INDIRECT COST],
        D.FUNDING,D.CITY,D.[STATE OR COUNTRY NAME],D.[INSTITUTION TYPE],D.YEAR,D.MONTH,
        D.FULL_LOC,D.CODE,COUNTY_FIPS,C.COUNTY_NAME,C.LAT,C.LNG
        FROM 
                (SELECT LNG,LAT,COUNTY_NAME,CITY,COUNTY_FIPS,STATE_ID
                FROM city) AS C
        LEFT JOIN decade as D
        ON D.CITY = C.CITY
        WHERE D.YEAR IN (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021) AND MONTH IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11) AND D.[FUNDING MECHANISM] IN ('None', 'Research Institutes', 'Domestic Higher Education', 'Independent Hospitals') AND D.[INSTITUTION TYPE] IN ('RPGs - SBIR/STTR', 'RPGs - Non SBIR/STTR', 'R&D Contracts', 'Other Research-Related', 'Training - Individual', 'Training - Institutional', 'Research Centers', 'Other', 'Construction')
        LIMIT 10
"""

In [50]:

conn = sqlite3.connect("data.db")
cursor = conn.cursor()

print(cursor.execute(cmd).fetchall())
conn.close()

[]
